In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler

from tot.datasets.dataset import Dataset
from tot.models.models_simple import RandomForestModel, LinearRegressionModel
from tot.models.models_neuralprophet import NeuralProphetModel
from tot.benchmark import SimpleBenchmark

In [2]:
data_location = "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/"
df_ercot = pd.read_csv(data_location + "multivariate/load_ercot_regions.csv")

In [3]:
regions = list(df_ercot)[1:]

In [4]:
df_global = pd.DataFrame()
for col in regions:
    aux = df_ercot[["ds", col]].copy(deep=True)  
    aux = aux.copy(deep=True) 
    aux = aux.iloc[:26301, :].copy(deep=True) 
    aux = aux.rename(columns={col: "y"})
    aux["ID"] = col
    df_global = pd.concat((df_global, aux))

In [5]:
dataset_list = [
    Dataset(df=df_global, name="df_global", freq="H"),
]

In [6]:
N_FORECASTS = 1
LR = 0.01
EPOCHS = 20

In [7]:
model_classes_and_params = [
    (RandomForestModel, {"n_forecasts": N_FORECASTS, "lags": 24, "output_chunk_length": N_FORECASTS}),
    (NeuralProphetModel, {
        "n_lags": 24,
        "n_forecasts": N_FORECASTS,
        "learning_rate": LR,
        "epochs": EPOCHS})
]

In [8]:
benchmark = SimpleBenchmark(
    model_classes_and_params=model_classes_and_params,
    datasets=dataset_list,
    metrics=["MAPE"],
    test_percentage=0.25,
)

In [ ]:
results_train, results_test = benchmark.run(verbose=True)

3 missing values in column y were detected in total. 
3 missing values in column y were detected in total. 
3 missing values in column y were detected in total. 
3 missing values in column y were detected in total. 
3 missing values in column y were detected in total. 
3 missing values in column y were detected in total. 
3 missing values in column y were detected in total. 
3 missing values in column y were detected in total. 


  0%|          | 0/19704 [00:00<?, ?it/s]

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
def plot_forecast(df, regions=regions):
    fig = go.Figure()

    for region in regions:
        fig.add_trace(
            go.Scatter(
                name=region + ' predicted', 
                x=df[df['ID'] == region]['ds'], 
                y=df[df['ID'] == region]['yhat1']))
        fig.add_trace(
            go.Scatter(
                name=region + ' actual', 
                mode="markers",
                x=df[df['ID'] == region]['ds'], 
                y=df[df['ID'] == region]['y']))
    fig.show()

In [ ]:
plot_forecast(benchmark.fcst_test[0], regions=["NORTH_C", "WEST"])

In [ ]:
plot_forecast(benchmark.fcst_test[1], regions=["NORTH_C", "WEST"])